In [1]:
import tensorflow as tf

from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2

# from spektral.layers import GraphConv
# GRaphConv is deprecated, use GCNConv or GCSConv instead
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.utils import normalized_laplacian
from spektral.layers import GCSConv  # as GraphConv
from spektral.layers import GINConv # as GraphConv
from spektral.layers import GCNConv  # as GraphConv

from spektral.utils.convolution import gcn_filter  # For GCNConv
from spektral.utils.convolution import normalized_adjacency  # For GCSConv

import numpy as np
import scipy.sparse
import pandas as pd
import pathlib
import os
import json

from sklearn.model_selection import train_test_split

from utils import *

In [2]:
# Parameters
l2_reg = 5e-4  # Regularization rate for l2
learning_rate = 1e-3  # Learning rate for SGD
batch_size = 32  # Batch size
epochs = 5  # Number of training epochs
es_patience = 200  # Patience fot early stopping

In [3]:
current_path = pathlib.Path().absolute()
path = current_path.parent

In [4]:
angry_path = path / 'angry_meshpoints'
disgusted_path = path / 'disgusted_meshpoints'
happy_path = path / 'happy_meshpoints'
neutral_path = path / 'neutral_meshpoints'
sad_path = path / 'sad_meshpoints'
surprised_path = path / 'surprised_meshpoints'

path_list = [angry_path, disgusted_path, happy_path, neutral_path, sad_path, surprised_path]

In [5]:
def extrai_dist(path_list, meshs_list, limit=0.1):
    for path in path_list:
        quant_files = len(os.listdir(path))
        count = 0
        for file in os.listdir(path):
            if (count / quant_files) > limit:
                break
            file_path = path / file
            count += 1
            with open(file_path, 'r') as f:
                data = json.load(f)
                meshs_list[path_list.index(path)].append(data)

In [6]:
meshs_surprised = []
meshs_disgusted = []
meshs_happy = []
meshs_neutral = []
meshs_sad = []
meshs_angry = []

meshs_list = [meshs_angry, meshs_disgusted, meshs_happy, meshs_neutral, meshs_sad, meshs_surprised]

extrai_dist(path_list, meshs_list, 0.1)

print(len(meshs_angry))
print(len(meshs_disgusted))
print(len(meshs_happy))
print(len(meshs_neutral))
print(len(meshs_sad))
print(len(meshs_surprised))

446
161
810
687
742
402


In [7]:
# meshs_list

In [8]:
# import concat from tensorflow
from tensorflow import concat
tensor_mesh = concat(meshs_list, axis=0)

In [9]:
tensor_mesh.shape

TensorShape([3248, 478, 2])

In [10]:
# target list
target_list = []
for i in range(6):
    target_list.append(np.full(len(meshs_list[i]), i))

target_list = concat(target_list, axis=0)


In [11]:
target_list.shape

TensorShape([3248])

In [12]:
n_out = 6  # Number of classes
N = tensor_mesh.shape[-2]  # Number of nodes in the graphs
F = tensor_mesh.shape[-1]  # Original feature dimensionality

In [13]:
N, F

(478, 2)

# Até aqui está exatamente como o professor sugeriu

## um tensor com 3 dimensões: (número de amostras, número de vértices, número de atributos)
## Outro com o target: (classe de cada amostra)

# GINConv Model

https://graphneural.network/layers/convolution/#ginconv

In [14]:
adj = tf.sparse.from_dense(tf.convert_to_tensor(get_mediapipe_adjacency_matrix()))

X_in = Input(shape=(N, F))

graph_conv_1 = GINConv(32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True)([X_in, adj])
graph_conv_2 = GINConv(32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True)([graph_conv_1, adj])
flatten = Flatten()(graph_conv_2)
fc = Dense(512, activation="relu")(flatten)
output = Dense(n_out, activation="softmax")(fc)

In [15]:
# Build model
model = Model(inputs=X_in, outputs=output)
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"]
)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 478, 2)]          0         
                                                                 
 gin_conv (GINConv)          (None, 478, 32)           97        
                                                                 
 gin_conv_1 (GINConv)        (None, 478, 32)           1057      
                                                                 
 flatten (Flatten)           (None, 15296)             0         
                                                                 
 dense (Dense)               (None, 512)               7832064   
                                                                 
 dense_1 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 7836296 (29.89 MB)
Trainable params: 7836296 (2

# Still need to figure out how to use the GINConv model

In [17]:
# o tensor_mesh é um tensor de dimensão 3
# o sparse_matrix é um tensor de dimensão 2
# o target_list é um tensor de dimensão 1

# Train model
model.fit(
    tensor_mesh,
    target_list,
    batch_size=batch_size,
    validation_split=0.2,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=es_patience)],
)




Epoch 1/5
82/82 [==============================] - 6s 62ms/step - loss: 1092.3715 - acc: 0.2502 - val_loss: 501.1785 - val_acc: 0.0000e+00
Epoch 2/5
82/82 [==============================] - 5s 56ms/step - loss: 78.6376 - acc: 0.2460 - val_loss: 343.1255 - val_acc: 0.0385
Epoch 3/5
82/82 [==============================] - 5s 56ms/step - loss: 25.7554 - acc: 0.2691 - val_loss: 223.1945 - val_acc: 0.0108
Epoch 4/5
82/82 [==============================] - 4s 55ms/step - loss: 30.9525 - acc: 0.2537 - val_loss: 30.2374 - val_acc: 0.0123
Epoch 5/5
82/82 [==============================] - 5s 56ms/step - loss: 4.4100 - acc: 0.3122 - val_loss: 23.9461 - val_acc: 0.1092
